In [ ]:
import requests
import pandas as pd

BASE_URL = "http://name.tr1-g5.dam.inspedralbes.cat:21787"  

def obtener_detalles_clientes():
    """
    Obtiene los detalles de los clientes desde la API y retorna un DataFrame con el nombre y apellido de cada cliente.
    """
    # Realiza la solicitud GET para obtener todos los usuarios
    response = requests.get(f"{BASE_URL}/user")
    if response.status_code != 200:
        print("Error al obtener los usuarios:", response.text)
        return None
    
    # Convertir la respuesta a un DataFrame
    users = response.json()
    user_df = pd.DataFrame(users)
    
    # Extraer los nombres completos (nombre + apellido) en una nueva columna
    user_df['full_name'] = user_df['firstName'] + ' ' + user_df['lastName']
    
    return user_df[['id', 'full_name']]  # Solo devolver id y el nombre completo

def obtener_promedio_diners_venta():
    """
    Obtiene los pedidos y calcula el promedio de dinero gastado por cliente, además agrega las columnas solicitadas.
    """
    # Realiza la solicitud GET para obtener todos los pedidos
    response = requests.get(f"{BASE_URL}/orders")
    if response.status_code != 200:
        print("Error al obtener los pedidos:", response.text)
        return
    
    # Convertir la respuesta a un DataFrame
    orders = response.json()
    df_orders = pd.DataFrame(orders)
    
    # Asegurarse de que la columna 'total' sea numérica
    df_orders['total'] = pd.to_numeric(df_orders['total'], errors='coerce')  # 'coerce' convierte los valores no numéricos a NaN
    
    # Eliminar filas con NaN en la columna 'total'
    df_orders = df_orders.dropna(subset=['total'])
    
    # Calcular el total de dinero gastado por cliente y el número de ventas
    df_sales = df_orders.groupby('userId').agg(
        Diners_per_Venda=('total', 'mean'),   # Promedio de dinero gastado por venta
        Ventas=('total', 'count'),            # Número de ventas de cada cliente
        Total_gastado=('total', 'sum')       # Total de dinero gastado por cliente
    ).reset_index()
    
    # Limitar los decimales de 'Diners_per_Venda' a dos
    df_sales['Diners_per_Venda'] = df_sales['Diners_per_Venda'].round(2)
    
    # Obtener detalles de los clientes (nombre completo)
    user_df = obtener_detalles_clientes()
    
    # Unir los datos de ventas con los detalles de los clientes
    df_sales = df_sales.merge(user_df, how='left', left_on='userId', right_on='id')
    
    # Eliminar la columna 'id' que ya no es necesaria
    df_sales = df_sales.drop(columns=['id'])
    
    # Agregar la columna de moneda
    df_sales['Moneda'] = '€'
    
    # Formatear el 'ID' para que tenga al menos dos dígitos, pero no recorte los números mayores a dos dígitos
    df_sales['ID'] = df_sales['userId'].apply(lambda x: f"{x:02d}" if x < 10 else str(x))
    
    # Reordenar las columnas y renombrar
    df_sales = df_sales[['ID', 'full_name', 'Ventas', 'Total_gastado', 'Diners_per_Venda', 'Moneda']]
    df_sales.columns = ['ID', 'Clients', 'Ventas', 'Diners', 'Diners/Venda', 'Moneda']
    
    # Mostrar los datos de ventas por cliente
    print("\nDatos de ventas por cliente:")
    print(df_sales)
    
    return df_sales

# Ejecutar la función y mostrar el DataFrame
df_promedio = obtener_promedio_diners_venta()



Datos de ventas por cliente:
   ID      Clients  Ventas  Diners  Diners/Venda Moneda
0  01     John Doe       3  124.95         41.65      €
1  02   Jane Smith       2  179.98         89.99      €
2  03  Alice Brown       2   95.97         47.98      €
3  04  Bob Johnson       1   89.98         89.98      €
